In [23]:
# Install required libraries
! pip install -U sentence-transformers
! pip install rouge-score
! pip install datasets

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


**Import Libraries.**

In [24]:
import nltk
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer 
from nltk.cluster import KMeansClusterer
from datasets import load_dataset
from rouge_score import rouge_scorer

**Initialize the Sentence Transformer with STS (Sentence Text Similarity) model.**

In [25]:
model = SentenceTransformer('stsb-roberta-base')

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


**Loading the Multi_News Dataset from Huggingface** 

In [26]:
# Load the dataset
dataset = load_dataset("alexfabbri/multi_news", split='train')

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for alexfabbri/multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/alexfabbri/multi_news
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [27]:
article = dataset[0]['document']
reference_summary = dataset[0]['summary']

**Now will convert our article into a list of sentences using nltk tokenizer.**

In [28]:
# Download punkt for sentence tokenization
nltk.download('punkt')

# Tokenize the article into sentences
sentences = nltk.sent_tokenize(article)

# Strip white spaces (leading & trailing)
sentences = [sentence.strip() for sentence in sentences]

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Check the tokenized sentences.**

In [29]:
sentences

['National Archives \n \n Yes, it’s that time again, folks.',
 'It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs.',
 'A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month.',
 'Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February.',
 'The unemployment rate is expected to hold steady at 8.3%.',
 'Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires.',
 'Feel free to weigh-in yourself, via the comments section.',
 'And while you’re here, why don’t you sign up to follow us on Twitter.',
 'Enjoy the show.',
 '||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. ec

**Convert above list in pandas dataframe**

In [30]:
# Create a DataFrame with the sentences
df = pd.DataFrame(sentences, columns=['sentences'])
df.head()

,sentences
0,"National Archives \n \n Yes, it’s that time ag..."
1,"It’s the first Friday of the month, when for o..."
2,A fresh update on the U.S. employment situatio...
3,"Expectations are for 203,000 new jobs to be cr..."
4,The unemployment rate is expected to hold stea...


In [31]:
print(f'Number of sentences in our article: {len(df)}')

Number of sentences in our article: 17


**Function that takes input as sentence & returns the dense vectors. Basically, this function converts the sentences into dense vectors.**

In [32]:
# Function to get sentence embeddings
def get_sent_embeddings(sent):
    embeddings = model.encode([sent])
    return embeddings[0]

**Apply above function to get embeddings for each sentence.**

In [33]:
# Get embeddings for each sentence
df['embeddings'] = df['sentences'].apply(get_sent_embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
df.head()

,sentences,embeddings
0,"National Archives \n \n Yes, it’s that time ag...","[0.8734846, -0.6017922, 0.71979344, 0.41540766..."
1,"It’s the first Friday of the month, when for o...","[-0.36445275, -0.44911194, 0.12615317, -1.0521..."
2,A fresh update on the U.S. employment situatio...,"[-0.2598091, -0.3050253, 0.5102481, -0.7973806..."
3,"Expectations are for 203,000 new jobs to be cr...","[0.18289545, -0.4410061, 0.12599786, 0.2161035..."
4,The unemployment rate is expected to hold stea...,"[-0.09652451, 0.0826514, -0.13469806, -0.56875..."


**Let's cluster text embeddings using nltk's KMeansCluster.**

In [35]:
# Set the number of clusters (summary sentences) and iterations
n_clusters = 10
iterations = 25

# Convert embeddings into numpy array
X = np.array(df['embeddings'].tolist())

# Perform clustering
kcluster = KMeansClusterer(n_clusters, distance=nltk.cluster.util.cosine_distance, repeats=iterations, avoid_empty_clusters=True)
assigned_clusters = kcluster.cluster(X, assign_clusters=True)

In [36]:
assigned_clusters

[8, 4, 4, 4, 0, 8, 6, 8, 9, 1, 3, 1, 2, 7, 4, 4, 5]

**Compute the distance between sentence embedding & centroid for each cluster.**

In [37]:
# Assign clusters and centroids to the DataFrame
df['Cluster'] = assigned_clusters
df['Centroid'] = df['Cluster'].apply(lambda x: kcluster.means()[x])

In [38]:
df.head()

,sentences,embeddings,Cluster,Centroid
0,"National Archives \n \n Yes, it’s that time ag...","[0.8734846, -0.6017922, 0.71979344, 0.41540766...",8,"[-0.017499462, -0.37744236, 0.35663563, 0.1651..."
1,"It’s the first Friday of the month, when for o...","[-0.36445275, -0.44911194, 0.12615317, -1.0521...",4,"[-0.3033929, -0.36989632, 0.1422538, -0.906370..."
2,A fresh update on the U.S. employment situatio...,"[-0.2598091, -0.3050253, 0.5102481, -0.7973806...",4,"[-0.3033929, -0.36989632, 0.1422538, -0.906370..."
3,"Expectations are for 203,000 new jobs to be cr...","[0.18289545, -0.4410061, 0.12599786, 0.2161035...",4,"[-0.3033929, -0.36989632, 0.1422538, -0.906370..."
4,The unemployment rate is expected to hold stea...,"[-0.09652451, 0.0826514, -0.13469806, -0.56875...",0,"[-0.09652451, 0.0826514, -0.13469806, -0.56875..."


**To compute the distance, we will use scipy's distance_matrix function**

In [39]:
# Function to calculate distance from centroid
from scipy.spatial import distance_matrix
def distance_from_centroid(row):
    dist_matrix = distance_matrix([row['embeddings']], [row['Centroid'].tolist()])[0][0]
    return dist_matrix

In [40]:
# Calculate distance from centroid for each sentence
df['distance_from_centroid'] = df.apply(distance_from_centroid, axis=1)
df.head()

,sentences,embeddings,Cluster,Centroid,distance_from_centroid
0,"National Archives \n \n Yes, it’s that time ag...","[0.8734846, -0.6017922, 0.71979344, 0.41540766...",8,"[-0.017499462, -0.37744236, 0.35663563, 0.1651...",13.637725
1,"It’s the first Friday of the month, when for o...","[-0.36445275, -0.44911194, 0.12615317, -1.0521...",4,"[-0.3033929, -0.36989632, 0.1422538, -0.906370...",11.326405
2,A fresh update on the U.S. employment situatio...,"[-0.2598091, -0.3050253, 0.5102481, -0.7973806...",4,"[-0.3033929, -0.36989632, 0.1422538, -0.906370...",10.189020
3,"Expectations are for 203,000 new jobs to be cr...","[0.18289545, -0.4410061, 0.12599786, 0.2161035...",4,"[-0.3033929, -0.36989632, 0.1422538, -0.906370...",11.940935
4,The unemployment rate is expected to hold stea...,"[-0.09652451, 0.0826514, -0.13469806, -0.56875...",0,"[-0.09652451, 0.0826514, -0.13469806, -0.56875...",0.000000


**The final step is to generate summary. This can be done by following steps:**

**1. Group the sentences based on Cluster column.**

**2. Sort the group ascending order based on distance_from_centroid column & select the first row (sentence having miimum distance from centroid).**

**3. Sort the sentences based on their sequence in the original text.**

In [41]:
# Sort sentences by distance from centroid and select the closest sentence from each cluster
sents = df.sort_values(by='distance_from_centroid', ascending=True).groupby('Cluster')
sents = sents.head(1)['sentences'].tolist()
sents

['Feel free to weigh-in yourself, via the comments section.',
 'The unemployment rate is expected to hold steady at 8.3%.',
 'The unemployment rate dipped, but mostly because more Americans stopped looking for work.',
 'Enjoy the show.',
 'The unemployment rate fell to 8.2 percent, the lowest since January 2009.',
 'The rate dropped because fewer people searched for jobs.',
 'But Federal Reserve Chairman Ben Bernanke has cautioned that the current hiring pace is unlikely to continue without more consumer spending.',
 '||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be growing fast enough to sustain robust job growth.',
 'A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month.',
 'And while you’re here, why don’t you sign up to follow us on Twitter.']

In [42]:
# Create the final summary
summary = ' '.join(sents)
print("Generated Summary:")
print(summary)

Generated Summary:
Feel free to weigh-in yourself, via the comments section. The unemployment rate is expected to hold steady at 8.3%. The unemployment rate dipped, but mostly because more Americans stopped looking for work. Enjoy the show. The unemployment rate fell to 8.2 percent, the lowest since January 2009. The rate dropped because fewer people searched for jobs. But Federal Reserve Chairman Ben Bernanke has cautioned that the current hiring pace is unlikely to continue without more consumer spending. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be growing fast enough to sustain robust job growth. A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. And while you’re here, why don’t you sign up to follow us on Twitter.


**Finding the Rogue Scores**

In [43]:
# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, summary)

In [44]:
# Print ROGUE scores
rouge1_score = scores['rouge1'].fmeasure
rouge2_score = scores['rouge2'].fmeasure
rougeL_score = scores['rougeL'].fmeasure

print("ROUGE-1 Score: ", rouge1_score)
print("ROUGE-2 Score: ", rouge2_score)
print("ROUGE-L Score: ", rougeL_score)

ROUGE-1 Score:  0.3666666666666667
ROUGE-2 Score:  0.1512605042016807
ROUGE-L Score:  0.18333333333333335
